# Low cost GNSS Data quality

In this section we will check 


The data provided hee corresponds to 



In [17]:
import matplotlib.pyplot as plt

from roktools import rinex

In [55]:
df_geodetic = rinex.to_dataframe('../assets/SEYG00SYC_R_20140581500_05H_01S_MO.rnx')


In [56]:
df_ublox = rinex.to_dataframe('../assets/MTIC00ESP_R_20191221131_05H_01S_MO.rnx')

### Observable differences

Once we have loaded the RINEX files into DataFrames, we can perform some
basic checks on the differences between geodetic and affordable GNSS data, for
instance the type of observables generated by one or the other.


In [59]:
columns = ['constellation', 'channel']

# Use groupby() to group by the two columns and apply unique()
unique_combinations = df_geodetic.groupby(columns).size()

# Print the unique combinations
print(unique_combinations)

constellation            channel
ConstellationId.GALILEO  1X         18910
                         5X         18910
                         7X         18910
                         8X         18910
ConstellationId.GPS      1C         87774
                         2W         87774
                         2X         87774
                         5X         87774
ConstellationId.GLONASS  1C         54304
                         1P         54304
                         2C         54304
                         2P         54304
ConstellationId.SBAS     1C         25134
dtype: int64


In [60]:
# Use groupby() to group by the two columns and apply unique()
unique_combinations = df_ublox.groupby(columns).size()

# Print the unique combinations
print(unique_combinations)

constellation            channel
ConstellationId.BEIDOU   2I          95280
ConstellationId.GALILEO  1C         106261
                         7Q         106261
ConstellationId.GPS      1C         138731
                         2L         138731
ConstellationId.GLONASS  1C         111380
                         2C         111380
dtype: int64


In these examples, the geodetic receiver tracks various frequencies (GPS L1/L2/L5, Galileo E1/E5a/E5b/E5, ...) whereas the affordable receiver tracks typically two frequencies (GPS L1/L2, Galileo E1/E5b, ...)

Some other strenghts of affordable receivers:

- Availability of SNR and Doppler measurements (not always available in 30s or high rate CORS data)
- High rate up to 0.1s (or even higher) available for affordable measurements



### Code estimation: Detrended LI

A basic quality parameter we can check is the computation of the detrended ionospheric observable
observation. The ionospheric combination removes the geometry, while the detrending process 
removes the ionosphere as well as the biases (ambiguities and hardware biases).
The remaining terms is an approximation of the phase noise (if LI is used) 
or the code noise (if PI is used instead)

In [83]:
import pandas as pd
from roktools.gnss.types import ConstellationId, TrackingChannel

def compute_cmc(df: pd.DataFrame, constellation: ConstellationId, channel_a: TrackingChannel, channel_b: TrackingChannel) -> pd.DataFrame:
    """
    Compute the Code minus Carrier
    """

    # Create subsets of the DataFrame corresponding to the constellation and each of
    # the channels selected to build the ionospheric combination
    df_a = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_a))]
    df_b = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_b))]

    # Compute the wavelength of the two tracking channels
    wl_a = channel_a.get_wavelength(constellation)
    wl_b = channel_b.get_wavelength(constellation)
    
    # Use merge to join the two tables
    df_out = pd.merge(df_a, df_b, on=['epoch', 'sat'], how='inner', suffixes=('_a', '_b'))
    df_out['li_m'] = df_out['phase_a'] * wl_a - df_out['phase_b'] * wl_b
    df_out['pi_m'] = df_out['range_b'] - df_out['range_a']

    # Define a custom rolling function to calculate the trend
    def do_rolling_window(group, column, new_column):
        group[new_column] = group[column].rolling(window=30).mean()
        return group

    # Apply the custom rolling function within each group
    df_tmp = df_out.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'li_m', 'li_trend_m'))
    b = df_tmp.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'pi_m', 'pi_trend_m'))

    return b

def get_seconds_since_start(epochs:pd.Series) -> list:
    """
    Converts a pandas series of epochs into a list of seconds relative to the first epoch
    """
    return (epochs - epochs[0]).dt.total_seconds().tolist()


In [84]:

satsys = ConstellationId.GPS
ch_a = TrackingChannel(1, 'C')
ch_b = TrackingChannel(2, 'W')
df_geodetic_li = compute_cmc(df_geodetic, satsys, ch_a, ch_b)

ch_a = TrackingChannel(1, 'C')
ch_b = TrackingChannel(2, 'C')
df_ublox_li = compute_cmc(df_ublox, satsys, ch_a, ch_b)



ValueError: 'sat' is both an index level and a column label, which is ambiguous.

In [81]:
a = df_geodetic_li.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'li_m', 'li_trend_m'))

In [82]:
a.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'pi_m', 'pi_trend_m'))

,epoch,constellation_a,sat,channel_a,signal_a,range_a,phase_a,doppler_a,snr_a,flag_a,...,range_b,phase_b,doppler_b,snr_b,flag_b,slip_b,li_m,pi_m,li_trend_m,pi_trend_m
0,2014-02-27 15:00:00,ConstellationId.GPS,G01,1C,G011C,2.434418e+07,1.279295e+08,-425.574,39.2,00000000,...,2.434421e+07,9.968548e+07,NaN,18.6,00000000,0,-36.684348,25.582,NaN,NaN
1,2014-02-27 15:00:00,ConstellationId.GPS,G04,1C,G041C,2.382347e+07,1.251933e+08,1432.680,39.5,00000000,...,2.382349e+07,9.755329e+07,NaN,22.0,00000000,0,-17.711599,17.414,NaN,NaN
2,2014-02-27 15:00:00,ConstellationId.GPS,G05,1C,G051C,2.506722e+07,1.317292e+08,1549.809,37.0,00000000,...,2.506725e+07,1.026462e+08,NaN,20.4,00000000,0,-30.585518,26.824,NaN,NaN
3,2014-02-27 15:00:00,ConstellationId.GPS,G07,1C,G071C,2.346388e+07,1.233036e+08,1093.793,39.7,00000000,...,2.346390e+07,9.608087e+07,NaN,22.2,00000000,0,-40.325221,16.801,NaN,NaN
4,2014-02-27 15:00:00,ConstellationId.GPS,G08,1C,G081C,2.418957e+07,1.271171e+08,1103.039,38.1,00000000,...,2.418959e+07,9.905244e+07,NaN,20.4,00000000,0,-38.773982,20.829,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87769,2014-02-27 19:59:59,ConstellationId.GPS,G10,1C,G101C,2.304162e+07,1.210848e+08,-336.141,41.8,00000000,...,2.304163e+07,9.435189e+07,-261.965,27.0,00000000,0,-23.682447,11.836,-23.677770,12.178633
87770,2014-02-27 19:59:59,ConstellationId.GPS,G15,1C,G151C,2.285345e+07,1.200958e+08,-281.391,45.3,00000000,...,2.285346e+07,9.358119e+07,-219.472,33.2,00000000,0,-19.520759,12.805,-19.504749,12.351067
87771,2014-02-27 19:59:59,ConstellationId.GPS,G24,1C,G241C,2.167144e+07,1.138841e+08,2345.555,47.8,00000000,...,2.167145e+07,8.874101e+07,1827.621,38.9,00000000,0,-38.376173,15.293,-38.352176,15.273533
87772,2014-02-27 19:59:59,ConstellationId.GPS,G26,1C,G261C,2.301703e+07,1.209555e+08,-1661.457,46.6,00000000,...,2.301705e+07,9.425102e+07,-1294.960,31.8,00000000,0,-0.104817,11.543,-0.101347,11.360833


In [31]:

plt.close()
seconds = get_seconds_since_start(df_geodetic_li['epoch'])
plt.plot(seconds, df_geodetic_li['li_m'] - df_geodetic_li['li_trend_m'], '.')

seconds = get_seconds_since_start(df_ublox_li['epoch'])
plt.plot(seconds, df_ublox_li['li_m'] - df_ublox_li['li_trend_m'], '.')




In [46]:
(df_test['epoch'] - df_test['epoch'][0]).dt.total_seconds()

0            0.0
1            0.0
2            0.0
3            1.0
4            1.0
          ...   
18905    17997.0
18906    17998.0
18907    17998.0
18908    17999.0
18909    17999.0
Name: epoch, Length: 18910, dtype: float64